In [42]:
import os
import sys

if not os.path.exists('/kaggle/working/efficient-kan'):
    !git clone https://github.com/Blealtan/efficient-kan

if '/kaggle/working/efficient-kan/src' not in sys.path:
    sys.path.append('/kaggle/working/efficient-kan/src')
    
!pip install gdown

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy
import gc
from tqdm import tqdm
from transformers import AutoTokenizer

from efficient_kan import KANLinear
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

In [43]:
!git clone https://github.com/stopwords/vietnamese-stopwords
    
with open('/kaggle/working/vietnamese-stopwords/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stop_words = file.read()
    stop_words = stop_words.split('\n')

fatal: destination path 'vietnamese-stopwords' already exists and is not an empty directory.


In [44]:
def remove_stopwords(text, stop_words):
    words = text.split(' ')
    
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    return ' '.join(filtered_words)

## Demo newspaper

In [45]:
content = '''
Sa Pa, Phan Thiết đông nghịt khách trong khi Đà Lạt, Vũng Tàu và nhiều điểm du lịch khác đông nhưng không xảy ra tình trạng chen chúc trong dịp lễ 2/9.

Như dự báo trước lễ, Sa Pa là điểm thu hút khách bậc nhất miền Bắc với công suất phòng 80-90%. Trong ngày 31/8 và 1/9, du khách di chuyển trong phố khó khăn vì đường nhỏ, lượng xe cá nhân đổ lên đông.

Theo một số đơn vị tổ chức tour Sa Pa, ngày thường, xe chở khách chỉ mất khoảng 15 phút để di chuyển từ bãi xe trung tâm về khách sạn nhưng dịp này phải mất hơn 40 phút. Do nhiều tuyến đường cấm xe ôtô trong dịp lễ như đoạn đi bản Cát Cát, các đoàn khách phải di chuyển bằng xe điện tới gần đường vào bản, tiếp tục đi bộ rồi trung chuyển lên ôtô để đi tham quan. Một số khách thiếu kinh nghiệm, không đặt trước phòng phải quay xe về Lào Cai hoặc đến các bản lân cận lưu trú.

Khu du lịch Fansipan Legend, nơi trang hoàng bằng hàng nghìn lá cờ đỏ sao vàng chào mừng Quốc khánh, ghi nhận lượng khách trong hai ngày đầu lễ tăng 45% so với cùng kỳ. Theo đại diện khu du lịch, nhiều doanh nghiệp, đơn vị đã chọn lên đỉnh Fansipan để du lịch và tham gia các hoạt động lan tỏa tình yêu nước. Một số đoàn phải đổi lịch trình tham quan Fansipan từ sáng sang chiều để tránh cảnh đông đúc.

Thời tiết miền Bắc dịp lễ nắng đẹp, mát mẻ, tạo điều kiện cho các đoàn du lịch. Minh Thắng, hướng dẫn viên du lịch cung Mộc Châu - Ninh Bình - Hà Nội nói các điểm tham quan không bị quá tải. Mộc Châu tắc đường nhẹ trong ngày 1/9 do tổ chức lễ hội. Tràng An, Ninh Bình đông khách hơn nhưng các đoàn cũng không phải chờ đợi lâu. Khu vực Phong Nha, Ninh Bình sáng 2/9 đón lượng khách không đông như năm ngoái.

Trong hai ngày đầu nghỉ lễ Quốc khánh 2/9 (31/9 và 1/9), tổng khách du lịch đến các điểm tham quan, du lịch trên toàn tỉnh Quảng Ninh ước đạt 289.000 lượt, bằng 134% cùng kỳ năm 2023. Hạ Long thu hút khách nhất tỉnh với 67.700 lượt, tiếp đến là Móng Cái, Vân Đồn với 22.000 lượt khách.

Ở miền Trung, Khu di tích Quốc gia đặc biệt Kim Liên, nằm trên địa bàn xã Kim Liên huyện Nam Đàn, tỉnh Nghệ An, dự kiến đón 3.000 đoàn khách và hàng chục nghìn lượt khách tới thăm dịp 2/9. Đây là quần thể di tích gắn liền với tuổi thơ Chủ tịch Hồ Chí Minh, cách thành phố Vinh 15 km.

"Nhiều năm nay, dịp Quốc khánh tôi luôn đưa các con đến Kim Liên tham quan để cháu hiểu rõ hơn về cuộc đời, sự nghiệp của Chủ tịch Hồ Chí Minh, từ đó nỗ lực, cố gắng đạt thành tích tốt trong năm học mới", chị Thanh Thủy, 39 tuổi, đến từ huyện Thạch Hà, tỉnh Hà Tĩnh, nói. Ban quản lý khu cho biết di tích đã huy động tất cả cán bộ, nhân viên làm việc thông suốt kỳ nghỉ lễ để người dân, du khách khi về với quê Bác được phục vụ tốt nhất.

Tại Đà Nẵng và Hội An, hướng dẫn viên du lịch Huỳnh Trung nói các điểm du lịch không xảy ra cảnh chen chúc hay phải chờ đợi lâu. Tại khu du lịch Bà Nà Hills, khách lên xuống cáp treo không cần xếp hàng. Khách du lịch dịp này chủ yếu là người địa phương hoặc các vùng lân cận.

Nguyễn Tiến, hướng dẫn viên cung Di sản miền Trung của Du Lịch Việt, nói các điểm chùa Linh Ứng, bán đảo Sơn Trà, Ngũ Hành Sơn, Hội An, Bà Nà Hills, thánh địa La Vang trong hai ngày 1/9 và 2/9 không bị quá tải. Quãng đường từ Huế ra Phong Nha, Quảng Bình sáng 2/9 cũng thông thoáng.

Đại diện Khu du lịch Bà Nà Hills dự kiến lượng khách trong bốn ngày lễ tăng 25% so với cùng kỳ năm trước. Khu du lịch đã triển khai nhiều hoạt động đón khách như trang trí cầu Vàng bằng hàng trăm lá cờ Tổ quốc, tạo hình nhiều món ăn lấy ý tưởng từ Quốc kỳ, miễn phí tham quan xưởng bia thủ công cho những du khách mặc áo cờ đỏ sao vàng.

Tại Nha Trang, thời tiết đẹp nhưng lượng khách không quá đông. Ghi nhận tại cảng tàu du lịch Nha Trang sáng 1/9, nhiều khách đợi tàu lâu nhưng sáng 2/9 không xảy ra tình trạng chen chúc. Hướng dẫn viên địa phương Nhã Phương nói đây là tình trạng chung dịp 2/9 vì khách đã dồn đi trong dịp hè. Dịp lễ này, nhiều resort, villa ở Cam Ranh hết phòng, chủ yếu khách từ địa phương lân cận đi xe cá nhân tới lưu trú.

Ở Quy Nhơn và Phú Yên thời tiết nắng nóng, vắng khách, giao thông thuận tiện như ngày thường. Điểm nóng du lịch Kỳ Co - Eo Gió ở Quy Nhơn không đông trong sáng 2/9. Tại Tuy Hòa, thời tiết nóng nên khách không ra tắm biển vào buổi sáng, các điểm du lịch trong thành phố cũng chung tình trạng vắng vẻ.
'''
print(len(content))

4265


In [46]:
cleaned_content = remove_stopwords(content, stop_words)
print(len(cleaned_content))

2839


In [47]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [48]:
test_sequence = tokenizer([cleaned_content],
                                    padding='max_length',
                                    truncation=True,
                                    max_length=8000,
                                    return_tensors="pt"
                                )
print(test_sequence['input_ids'].shape)

torch.Size([1, 8000])


In [49]:
labels_dict = {
    "chính trị": 1,
    "xã hội": 2,
    "kinh tế": 3,
    "văn hóa": 4,
    "sức khỏe": 5,
    "pháp luật": 6,
    "thế giới": 7,
    "khcn": 8,
    "thể thao": 9,
    "giải trí": 10,
    "du lịch": 11,
    "giáo dục": 12
}

reverse_labels_dict = {v: k for k, v in labels_dict.items()}
print(reverse_labels_dict)

num_classes = 12

{1: 'chính trị', 2: 'xã hội', 3: 'kinh tế', 4: 'văn hóa', 5: 'sức khỏe', 6: 'pháp luật', 7: 'thế giới', 8: 'khcn', 9: 'thể thao', 10: 'giải trí', 11: 'du lịch', 12: 'giáo dục'}


## Define models and load weights

In [50]:
!gdown --folder https://drive.google.com/drive/folders/1iG4ubXgej2_KMAsrdIB8CpdGypKTJTrf

Retrieving folder contents
Processing file 12Q72qowIDOycscGrPEcCv6cgTkXQi7Qq 1d_cnn_best_loss_weigth.pth
Processing file 1NS8GCH_xGUY6YDjZel9c1IN83q4SqHFo bi_gru_best_loss_weigth.pth
Processing file 1gRO5HeiI-kTItd7uBZYleGd9-gsm5u22 bi_lstm_best_loss_weigth.pth
Processing file 1J-KVSRvwOrbaXQAHM9IwAKgHhR3eSTOl minibert_best_loss_weigth.pth
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=12Q72qowIDOycscGrPEcCv6cgTkXQi7Qq
To: /kaggle/working/weights/1d_cnn_best_loss_weigth.pth
100%|██████████████████████████████████████| 24.8M/24.8M [00:01<00:00, 16.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NS8GCH_xGUY6YDjZel9c1IN83q4SqHFo
To: /kaggle/working/weights/bi_gru_best_loss_weigth.pth
100%|███████████████████████████████████████| 11.0M/11.0M [00:00<00:00, 102MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gRO5HeiI-kTItd7uBZYleGd9-gsm5u22
To: /kaggle/working/w

In [51]:
class Block1(nn.Module):
    def __init__(self, in_channel, out_channel, seq_len):
        super().__init__()
        self.backbone = nn.Sequential(
            nn.Conv1d(in_channel, out_channel, 5, stride=1, dilation=2, padding=4),
            nn.LayerNorm([out_channel, seq_len]),
            nn.SiLU(),
            nn.Dropout(0.5)
        )
        
    def forward(self, x):
        return self.backbone(x)

# 1D CNN
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        st = 32
        
        self.input_embedding = nn.Embedding(64000, st)
        
        self.backbone = nn.Sequential(
            Block1(st, 2*st, 8000),
            nn.AvgPool1d(2),
            Block1(2*st, 4*st, 4000),
            nn.AvgPool1d(2),
            Block1(4*st, 8*st, 2000),
            nn.AvgPool1d(2),
            Block1(8*st, 8*st, 1000),
            nn.AdaptiveAvgPool1d(1)
        )
        
        self.classifier = KANLinear(8*st, num_classes)
        
    def forward(self, x):
        x = self.input_embedding(x)
        if (x.shape[0] != 1):
            x = x.squeeze()
        x = x.transpose(1, 2)
        x = self.backbone(x).squeeze()
        x = self.classifier(x)
        return x

model1 = Model1().to(device)
model1.load_state_dict(torch.load('/kaggle/working/weights/1d_cnn_best_loss_weigth.pth', map_location=device))
model1.eval()

/tmp/ipykernel_36/973584290.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load('/kaggle/working/weights/1d_cnn_best_loss_weigth.pth', map

Model1(
  (input_embedding): Embedding(64000, 32)
  (backbone): Sequential(
    (0): Block1(
      (backbone): Sequential(
        (0): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
        (1): LayerNorm((64, 8000), eps=1e-05, elementwise_affine=True)
        (2): SiLU()
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (2): Block1(
      (backbone): Sequential(
        (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
        (1): LayerNorm((128, 4000), eps=1e-05, elementwise_affine=True)
        (2): SiLU()
        (3): Dropout(p=0.5, inplace=False)
      )
    )
    (3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    (4): Block1(
      (backbone): Sequential(
        (0): Conv1d(128, 256, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
        (1): LayerNorm((256, 2000), eps=1e-05, elementwise_affine=True)
        (2): SiLU()

In [52]:
# Bi-LSTM
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        st = 32
        
        self.input_embedding = nn.Embedding(64000, st)
        
        # Reduce sequence length from 8000 to 1000
        self.conv = nn.Sequential(
            nn.Conv1d(st, st*2, 5, stride=2, padding=2),
            nn.LayerNorm([st*2, 4000]),
            nn.SiLU(),
            nn.AvgPool1d(8, stride=4, padding=2),
            nn.Dropout(0.5)
        )
        
        self.hidden_size = st*4
        self.num_layers = 1
        self.backbone = nn.LSTM(st*2, self.hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=True)
        
        self.classifier = KANLinear(self.hidden_size*2, num_classes)
        
    def forward(self, x):        
        x = self.input_embedding(x)
        if (x.shape[0] != 1):
            x = x.squeeze()
        
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = x.transpose(1, 2)
        
        h0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.backbone(x, (h0, c0))
            
        output = self.classifier(output[:, -1, :])
        return output
    
model2 = Model2().to(device)
model2.load_state_dict(torch.load('/kaggle/working/weights/bi_lstm_best_loss_weigth.pth', map_location=device))
model2.eval()

/tmp/ipykernel_36/1719540135.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load('/kaggle/working/weights/bi_lstm_best_loss_weigth.pth', m

Model2(
  (input_embedding): Embedding(64000, 32)
  (conv): Sequential(
    (0): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): LayerNorm((64, 4000), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
    (3): AvgPool1d(kernel_size=(8,), stride=(4,), padding=(2,))
    (4): Dropout(p=0.5, inplace=False)
  )
  (backbone): LSTM(64, 128, batch_first=True, bidirectional=True)
  (classifier): KANLinear(
    (base_activation): SiLU()
  )
)

In [53]:
# Bi-GRU
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        st = 32
        
        self.input_embedding = nn.Embedding(64000, st)
        
        # Reduce sequence length from 8000 to 1000
        self.conv = nn.Sequential(
            nn.Conv1d(st, st*2, 5, stride=2, padding=2),
            nn.LayerNorm([st*2, 4000]),
            nn.SiLU(),
            nn.AvgPool1d(8, stride=4, padding=2),
            nn.Dropout(0.5)
        )
        
        self.hidden_size = st*4
        self.num_layers = 1
        self.backbone = nn.GRU(st*2, self.hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=True)
        
        self.classifier = KANLinear(self.hidden_size*2, num_classes)
        
    def forward(self, x):        
        x = self.input_embedding(x)
        if (x.shape[0] != 1):
            x = x.squeeze()
        
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = x.transpose(1, 2)
        
        h0 = torch.zeros(2*self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        output, _ = self.backbone(x, h0)
            
        output = self.classifier(output[:, -1, :])
        return output
    
model3 = Model3().to(device)
model3.load_state_dict(torch.load('/kaggle/working/weights/bi_gru_best_loss_weigth.pth', map_location=device))
model3.eval()

/tmp/ipykernel_36/1390676918.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model3.load_state_dict(torch.load('/kaggle/working/weights/bi_gru_best_loss_weigth.pth', ma

Model3(
  (input_embedding): Embedding(64000, 32)
  (conv): Sequential(
    (0): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): LayerNorm((64, 4000), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
    (3): AvgPool1d(kernel_size=(8,), stride=(4,), padding=(2,))
    (4): Dropout(p=0.5, inplace=False)
  )
  (backbone): GRU(64, 128, batch_first=True, bidirectional=True)
  (classifier): KANLinear(
    (base_activation): SiLU()
  )
)

In [54]:
# Kernel function
class Phi(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return F.elu(x) + 1

class AttentionLayer(nn.Module): # Linear Attention using Kernel
    def __init__(self, d_model=512, num_heads=8):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads

        self.W_query = nn.Linear(d_model, d_model)
        self.W_key = nn.Linear(d_model, d_model)
        self.W_value = nn.Linear(d_model, d_model)
        
        self.phi = Phi()
        
        self.out_proj = KANLinear(d_model, d_model)

    def forward(self, x):
        bs, seq_len, d_model = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Unroll last dim: (bs, seq_len, d_model) -> (bs, num_tokens, seq_len, num_head, head_dim)
        keys = keys.view(bs, seq_len, self.num_heads, self.head_dim)
        values = values.view(bs, seq_len, self.num_heads, self.head_dim)
        queries = queries.view(bs, seq_len, self.num_heads, self.head_dim)

        # Transpose: (bs, seq_len, num_heads, head_dim) -> (bs, num_heads, seq_len, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_weights = self.phi(keys.transpose(-1, -2)) @ values
        attn_weights = self.phi(queries) @ attn_weights

        # Combine heads, where self.d_model = self.num_heads * self.head_dim
        attn_weights = attn_weights.reshape(bs, seq_len, self.d_model)
        out = self.out_proj(attn_weights)

        return out
    
class EncoderLayer(nn.Module): # Pre-layernorm
    def __init__(self, d_model=512, num_heads=8, dropout=0.2):
        super().__init__()
        
        self.attention = AttentionLayer(d_model, num_heads)
        self.ff = KANLinear(d_model, d_model)
        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        tmp = self.ln1(x)
        tmp = self.attention(tmp)
        tmp = self.dropout(tmp)
        x = x + tmp
        
        tmp = self.ln2(x)
        tmp = self.ff(tmp)
        tmp = self.dropout(tmp)
        x = x + tmp
        return x
    
# MiniBERT
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        st = 32
        
        self.input_embedding = nn.Embedding(64000, st)
        
        # Reduce sequence length from 8000 to 1000
        self.conv = nn.Sequential(
            nn.Conv1d(st, st*2, 5, stride=2, padding=2),
            nn.LayerNorm([st*2, 4000]),
            nn.SiLU(),
            nn.AvgPool1d(8, stride=4, padding=2)
        )
        
        self.upsampling = nn.Sequential(
            KANLinear(st*2, st*4),
            nn.Dropout(0.4)
        )
        
        self.pos_emb = nn.Embedding(1000, st*4)
        position = torch.arange(0, 1000).unsqueeze(0)
        self.register_buffer('position', position)
        
        self.backbone = EncoderLayer(d_model=st*4, num_heads=4, dropout=0.4)
        self.lastnorm = nn.LayerNorm(st*4)
        
        self.classifier = KANLinear(st*4, num_classes)
        
    def forward(self, x):        
        x = self.input_embedding(x)
        if (x.shape[0] != 1):
            x = x.squeeze()
        
        x = x.transpose(1, 2)
        x = self.conv(x)
        x = x.transpose(1, 2)
        x = self.upsampling(x)
        
        x = x + self.pos_emb(self.position)
        
        output = self.backbone(x)
        output = self.lastnorm(output)
            
        output = self.classifier(output[:, 0, :])
        return output
    
model4 = Model4().to(device)
model4.load_state_dict(torch.load('/kaggle/working/weights/minibert_best_loss_weigth.pth', map_location=device))
model4.eval()

/tmp/ipykernel_36/3495480299.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model4.load_state_dict(torch.load('/kaggle/working/weights/minibert_best_loss_weigth.pth',

Model4(
  (input_embedding): Embedding(64000, 32)
  (conv): Sequential(
    (0): Conv1d(32, 64, kernel_size=(5,), stride=(2,), padding=(2,))
    (1): LayerNorm((64, 4000), eps=1e-05, elementwise_affine=True)
    (2): SiLU()
    (3): AvgPool1d(kernel_size=(8,), stride=(4,), padding=(2,))
  )
  (upsampling): Sequential(
    (0): KANLinear(
      (base_activation): SiLU()
    )
    (1): Dropout(p=0.4, inplace=False)
  )
  (pos_emb): Embedding(1000, 128)
  (backbone): EncoderLayer(
    (attention): AttentionLayer(
      (W_query): Linear(in_features=128, out_features=128, bias=True)
      (W_key): Linear(in_features=128, out_features=128, bias=True)
      (W_value): Linear(in_features=128, out_features=128, bias=True)
      (phi): Phi()
      (out_proj): KANLinear(
        (base_activation): SiLU()
      )
    )
    (ff): KANLinear(
      (base_activation): SiLU()
    )
    (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (ln2): LayerNorm((128,), eps=1e-05, elementwise_affi

## Testing

In [55]:
print(content)


Sa Pa, Phan Thiết đông nghịt khách trong khi Đà Lạt, Vũng Tàu và nhiều điểm du lịch khác đông nhưng không xảy ra tình trạng chen chúc trong dịp lễ 2/9.

Như dự báo trước lễ, Sa Pa là điểm thu hút khách bậc nhất miền Bắc với công suất phòng 80-90%. Trong ngày 31/8 và 1/9, du khách di chuyển trong phố khó khăn vì đường nhỏ, lượng xe cá nhân đổ lên đông.

Theo một số đơn vị tổ chức tour Sa Pa, ngày thường, xe chở khách chỉ mất khoảng 15 phút để di chuyển từ bãi xe trung tâm về khách sạn nhưng dịp này phải mất hơn 40 phút. Do nhiều tuyến đường cấm xe ôtô trong dịp lễ như đoạn đi bản Cát Cát, các đoàn khách phải di chuyển bằng xe điện tới gần đường vào bản, tiếp tục đi bộ rồi trung chuyển lên ôtô để đi tham quan. Một số khách thiếu kinh nghiệm, không đặt trước phòng phải quay xe về Lào Cai hoặc đến các bản lân cận lưu trú.

Khu du lịch Fansipan Legend, nơi trang hoàng bằng hàng nghìn lá cờ đỏ sao vàng chào mừng Quốc khánh, ghi nhận lượng khách trong hai ngày đầu lễ tăng 45% so với cùng kỳ.

In [56]:
y_pred = model1(test_sequence['input_ids'])
y_pred = torch.argmax(y_pred) + 1
y_pred = reverse_labels_dict[y_pred.item()]
print(y_pred)

du lịch


In [57]:
y_pred = model2(test_sequence['input_ids'])
y_pred = torch.argmax(y_pred) + 1
y_pred = reverse_labels_dict[y_pred.item()]
print(y_pred)

xã hội


In [58]:
y_pred = model3(test_sequence['input_ids'])
y_pred = torch.argmax(y_pred) + 1
y_pred = reverse_labels_dict[y_pred.item()]
print(y_pred)

du lịch


In [59]:
y_pred = model4(test_sequence['input_ids'])
y_pred = torch.argmax(y_pred) + 1
y_pred = reverse_labels_dict[y_pred.item()]
print(y_pred)

du lịch
